In [2]:
using ParallelTemperingMonteCarlo
import Pkg;Pkg.add("BenchmarkTools")
using Benchmarktools

   Resolving package versions...


    Updating `~/Documents/ParallelTemperingMonteCarlo.jl/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  No Changes to `~/Documents/ParallelTemperingMonteCarlo.jl/Manifest.toml`


ArgumentError: ArgumentError: Package Benchmarktools not found in current path.
- Run `import Pkg; Pkg.add("Benchmarktools")` to install the Benchmarktools package.